### Дообучение

_Идея:_ **Брать логи за последние 10 дней (нормально можно обучить) и замешивать в выборку юзерагенты из старых выборок которые входят в топ 200 старой выборки, но не входят в топ 200 новой.**

Для выборки по времени у класса ДщпЗфкіу реализована загрузка спец тренировочной выборки `parse_train_sample_by_time`

In [1]:
import pandas as pd
import numpy as np

import scipy.sparse
import sklearn.feature_extraction

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm
import platform

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

import os

%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/levabd/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### Load data to train

In [2]:
main_data = np.load('df/main_data.npy').tolist()
values_data = np.load('df/values_data.npy').tolist()
order_data = np.load('df/order_data.npy').tolist()

main_df = pd.DataFrame(main_data)

main_df

,device_class_code,ip,os_code,os_family_code,timestamp,ua_class_code,ua_family_code,ua_string,ua_version
0,desktop,95.181.252.91,windows_7,windows,1485900038,browser,chrome,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,chrome55.0.2883.87
1,smartphone,31.148.3.19,android_4_4,android,1485900079,mobile_browser,chrome_mobile,Mozilla/5.0 (Linux; Android 4.4.2; Zera S Buil...,chrome_mobile49.0.2623.105
2,desktop,188.162.183.66,windows_7,windows,1485900103,browser,firefox,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) G...,firefox51.0
3,desktop,37.144.52.103,windows_xp,windows,1485900105,browser,ie,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,ie8.0
4,desktop,91.201.178.51,windows_8_1,windows,1485900108,browser,opera,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,opera40.0.2308.81
...,...,...,...,...,...,...,...,...,...
571140,smartphone,89.237.103.208,android_6,android,1488585286,mobile_browser,chrome_mobile,Mozilla/5.0 (Linux; Android 6.0.1; SM-G925F Bu...,chrome_mobile56.0.2924.87
571141,desktop,141.0.12.169,windows_98,windows,1488585377,browser,opera,Mozilla/4.0 (Windows 98; US) Opera 12.16 [en],opera12.16
571142,desktop,131.253.24.147,windows_7,windows,1488585395,browser,ie,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,ie9.0
571143,desktop,176.195.122.100,windows_7,windows,1488585447,browser,ie,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,ie8.0


In [4]:
with open("df/group_labels/list_ua_version.txt", 'r') as f:
    list_ua_version = [line.rstrip('\n') for line in f]
    
with open("df/group_labels/list_ua_family_code.txt", 'r') as f:
    list_ua_family_code = [line.rstrip('\n') for line in f]

with open("df/group_labels/list_ua_class_code.txt", 'r') as f:
    list_ua_class_code = [line.rstrip('\n') for line in f]
    
with open("df/group_labels/list_os_code.txt", 'r') as f:
    list_os_code = [line.rstrip('\n') for line in f]
    
with open("df/group_labels/list_os_family_code.txt", 'r') as f:
    list_os_family_code = [line.rstrip('\n') for line in f]
    
with open("df/group_labels/list_device_class_code.txt", 'r') as f:
    list_device_class_code = [line.rstrip('\n') for line in f]

print("Device count: {}".format(len(list_device_class_code)))
print("Device platform family count: {}".format(len(list_os_family_code)))
print("Device platform count: {}".format(len(list_os_code)))
print("Device browser class count: {}".format(len(list_ua_class_code)))
print("Device browser family count: {}".format(len(list_ua_family_code)))
print("Device browser version count: {}".format(len(list_ua_version)))

Device count: 8
Device platform family count: 37
Device platform count: 109
Device browser class count: 10
Device browser family count: 193
Device browser version count: 5790


In [5]:
important_orders_keys_set = {
    'Upgrade-Insecure-Requests',
    'Accept', 
    'If-Modified-Since',
    'Host', 
    'Connection', 
    'User-Agent', 
    'From', 
    'Accept-Encoding' 
}

important_values_keys_set = {
    'Accept', 
    'Accept-Charset', 
    'Accept-Encoding'
}

In [6]:
from lib.parsers.logParser import LogParser

orders_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
values_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)

l_parser = LogParser(log_folder='Logs/')
l_parser.reassign_orders_values(order_data, values_data)

full_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=True)

  3%|▎         | 18149/571145 [00:00<00:03, 181093.82it/s]

Sparse dummy orders shape: 
(571145, 50)


100%|██████████| 571145/571145 [00:01<00:00, 366313.77it/s]


Sparse dummy values shape: 
(571145, 516)


### Save vectorizers

In [ ]:
import os
from sklearn.externals import joblib
from lib.helpers.fileSplitter import split_file

filename = 'cls/orders_vectorizer30.joblib.pkl'
_ = joblib.dump(orders_vectorizer, filename, compress=9)

print("Orders Vectorizer saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/orders_vectorizer30.joblib.pkl')

print('Splitted in {} files'.format(files_count))

filename = 'cls/values_vectorizer30.joblib.pkl'
_ = joblib.dump(orders_vectorizer, filename, compress=9)

print("Values Vectorizer saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/values_vectorizer30.joblib.pkl')

print('Splitted in {} files'.format(files_count))

### Train and on groups

#### Device_class_code

In [8]:
%%time

from sklearn.linear_model import LogisticRegression

clf_device_class_code = LogisticRegression(random_state=42, C=100)
clf_device_class_code.fit(full_sparce_dummy, main_df.device_class_code.fillna('NaN'))

CPU times: user 4min 23s, sys: 460 ms, total: 4min 24s
Wall time: 5min 13s


In [ ]:
filename = 'cls/device_class_code_logreg_cls.joblib.pkl'
_ = joblib.dump(clf_device_class_code, filename, compress=9)

print("Model saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/device_class_code_logreg_cls.joblib.pkl')

print('Splitted in {} files'.format(files_count))

#### OS_family_code

In [9]:
%%time

clf_os_family_code = LogisticRegression(random_state=42, C=100)
clf_os_family_code.fit(full_sparce_dummy, main_df.os_family_code.fillna('NaN'))

CPU times: user 6min 41s, sys: 556 ms, total: 6min 41s
Wall time: 10min 35s


In [ ]:
filename = 'cls/os_family_code_logreg_cls.joblib.pkl'
_ = joblib.dump(clf_device_class_code, filename, compress=9)

print("Model saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/os_family_code_logreg_cls.joblib.pkl')

print('Splitted in {} files'.format(files_count))

#### OS_code

In [10]:
%%time

clf_os_code = LogisticRegression(random_state=42, C=100)
clf_os_code.fit(full_sparce_dummy, main_df.os_code.fillna('NaN'))

CPU times: user 28min 41s, sys: 1.56 s, total: 28min 43s
Wall time: 32min 20s


In [ ]:
filename = 'cls/os_code_logreg_cls.joblib.pkl'
_ = joblib.dump(clf_device_class_code, filename, compress=9)

print("Model saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/os_code_logreg_cls.joblib.pkl')

print('Splitted in {} files'.format(files_count))

#### Browser class_code

In [ ]:
%%time

clf_ua_class_code = LogisticRegression(random_state=42, C=100)
clf_ua_class_code.fit(full_sparce_dummy, main_df.ua_class_code.fillna('NaN'))

In [ ]:
filename = 'cls/ua_class_code_logreg_cls.joblib.pkl'
_ = joblib.dump(clf_device_class_code, filename, compress=9)

print("Model saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/ua_class_code_logreg_cls.joblib.pkl')

print('Splitted in {} files'.format(files_count))

#### Browser family_code

In [ ]:
%%time

clf_ua_family_code = LogisticRegression(random_state=42, C=100)
clf_ua_family_code.fit(full_sparce_dummy, main_df.ua_family_code.fillna('NaN'))

CPU times: user 26min 20s, sys: 1.42 s, total: 26min 22s
Wall time: 30min 44s


In [ ]:
filename = 'cls/ua_family_code_logreg_cls.joblib.pkl'
_ = joblib.dump(clf_device_class_code, filename, compress=9)

print("Model saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/ua_family_code_logreg_cls.joblib.pkl')

print('Splitted in {} files'.format(files_count))

#### Browser version

In [ ]:
%%time

clf_ua_version = LogisticRegression(random_state=42, C=100)
clf_ua_version.fit(full_sparce_dummy, main_df.ua_version.fillna('NaN'))

In [ ]:
filename = 'cls/ua_version_logreg_cls.joblib.pkl'
_ = joblib.dump(clf_device_class_code, filename, compress=9)

print("Model saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/ua_version_logreg_cls.joblib.pkl')

print('Splitted in {} files'.format(files_count))

### Load test data and free train data

In [ ]:
del(main_df)
del(main_data)
del(values_data)
del(order_data)
del(full_sparce_dummy)

In [ ]:
main2_data = np.load('df/main2_data.npy').tolist()
values2_data = np.load('df/values2_data.npy').tolist()
order2_data = np.load('df/order2_data.npy').tolist()

main2_df = pd.DataFrame(main2_data)

main2_df

### Load vectorizers if needed

In [ ]:
infiles = [
    'parted-cls/orders_vectorizer30.joblib.pkl.0',
    'parted-cls/orders_vectorizer30.joblib.pkl.1',
    'parted-cls/orders_vectorizer30.joblib.pkl.2',
    'parted-cls/orders_vectorizer30.joblib.pkl.3',
    'parted-cls/orders_vectorizer30.joblib.pkl.4',
    'parted-cls/orders_vectorizer30.joblib.pkl.5'
]

import os
from sklearn.externals import joblib
from lib.helpers.fileSplitter import cat_files

cat_files(infiles, 'cls/orders_vectorizer30.joblib.pkl')

filename = 'cls/orders_vectorizer30.joblib.pkl'
orders_vectorizer = joblib.load(filename)

infiles = [
    'parted-cls/values_vectorizer30.joblib.pkl.0',
    'parted-cls/values_vectorizer30.joblib.pkl.1',
    'parted-cls/values_vectorizer30.joblib.pkl.2',
    'parted-cls/values_vectorizer30.joblib.pkl.3',
    'parted-cls/orders_vectorizer30.joblib.pkl.4',
    'parted-cls/orders_vectorizer30.joblib.pkl.5'
]

cat_files(infiles, 'cls/values_vectorizer30.joblib.pkl')

filename = 'cls/values_vectorizer30.joblib.pkl'
values_vectorizer = joblib.load(filename)

In [ ]:
l_parser = LogParser(log_folder='Logs/')
l_parser.reassign_orders_values(order2_data, values2_data)

full2_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=False)

### Get prediction probabilities by groups

### Load classifyers models if needed

In [ ]:
infiles = [
    'parted-cls/device_class_code_logreg_cls.joblib.pkl.0',
    'parted-cls/device_class_code_logreg_cls.joblib.pkl.1',
    'parted-cls/device_class_code_logreg_cls.joblib.pkl.2',
    'parted-cls/device_class_code_logreg_cls.joblib.pkl.3',
    'parted-cls/device_class_code_logreg_cls.joblib.pkl.4',
    'parted-cls/device_class_code_logreg_cls.joblib.pkl.5'
]

cat_files(infiles, 'cls/device_class_code_logreg_cls.joblib.pkl')
clf_device_class_code = joblib.load('cls/device_class_code_logreg_cls.joblib.pkl')

infiles = [
    'parted-cls/os_family_code_logreg_cls.joblib.pkl.0',
    'parted-cls/os_family_code_logreg_cls.joblib.pkl.1',
    'parted-cls/os_family_code_logreg_cls.joblib.pkl.2',
    'parted-cls/os_family_code_logreg_cls.joblib.pkl.3',
    'parted-cls/os_family_code_logreg_cls.joblib.pkl.4',
    'parted-cls/os_family_code_logreg_cls.joblib.pkl.5'
]

cat_files(infiles, 'cls/os_family_code_logreg_cls.joblib.pkl')
clf_os_family_code = joblib.load('cls/os_family_code_logreg_cls.joblib.pkl')

infiles = [
    'parted-cls/os_code_logreg_cls.joblib.pkl.0',
    'parted-cls/os_code_logreg_cls.joblib.pkl.1',
    'parted-cls/os_code_logreg_cls.joblib.pkl.2',
    'parted-cls/os_code_logreg_cls.joblib.pkl.3',
    'parted-cls/os_code_logreg_cls.joblib.pkl.4',
    'parted-cls/os_code_logreg_cls.joblib.pkl.5'
]

cat_files(infiles, 'cls/os_code_logreg_cls.joblib.pkl')
clf_os_code = joblib.load('cls/os_code_logreg_cls.joblib.pkl')

infiles = [
    'parted-cls/ua_class_code_logreg_cls.joblib.pkl.0',
    'parted-cls/ua_class_code_logreg_cls.joblib.pkl.1',
    'parted-cls/ua_class_code_logreg_cls.joblib.pkl.2',
    'parted-cls/ua_class_code_logreg_cls.joblib.pkl.3',
    'parted-cls/ua_class_code_logreg_cls.joblib.pkl.4',
    'parted-cls/ua_class_code_logreg_cls.joblib.pkl.5'
]

cat_files(infiles, 'cls/ua_class_code_logreg_cls.joblib.pkl')
clf_ua_class_code = joblib.load('cls/ua_class_code_logreg_cls.joblib.pkl')

infiles = [
    'parted-cls/ua_family_code_logreg_cls.joblib.pkl.0',
    'parted-cls/ua_family_code_logreg_cls.joblib.pkl.1',
    'parted-cls/ua_family_code_logreg_cls.joblib.pkl.2',
    'parted-cls/ua_family_code_logreg_cls.joblib.pkl.3',
    'parted-cls/ua_family_code_logreg_cls.joblib.pkl.4',
    'parted-cls/ua_family_code_logreg_cls.joblib.pkl.5'
]

cat_files(infiles, 'cls/ua_family_code_logreg_cls.joblib.pkl')
clf_ua_family_code = joblib.load('cls/ua_family_code_logreg_cls.joblib.pkl')

infiles = [
    'parted-cls/ua_version_logreg_cls.joblib.pkl.0',
    'parted-cls/ua_version_logreg_cls.joblib.pkl.1',
    'parted-cls/ua_version_logreg_cls.joblib.pkl.2',
    'parted-cls/ua_version_logreg_cls.joblib.pkl.3',
    'parted-cls/ua_version_logreg_cls.joblib.pkl.4',
    'parted-cls/ua_version_logreg_cls.joblib.pkl.5'
]

cat_files(infiles, 'cls/ua_version_logreg_cls.joblib.pkl')
clf_ua_version = joblib.load('cls/ua_version_logreg_cls.joblib.pkl')

In [ ]:
pp_dc = clf_device_class_code.predict_proba(full2_sparce_dummy)
zip(clf_device_class_code.classes_, pp_dc[0])

In [ ]:
pp_device_class_code = pd.DataFrame(pp_dc, columns=clf_device_class_code.classes_)
pp_device_class_code

### Generate new human features based on probability of real class

In [ ]:
def single_class_proba(probas, labels, expected_class):
    """
    :param probas: prediction vector
    :param labels: labels of classification
    :param expected_class: label of real truth class answer
    :return: probability of truth class answer
    """
    for i, label in enumerate(labels):
        if label == expected_class:
            return probas[i]

In [ ]:
print(main2_df.device_class_code[0])
single_class_proba(pp_dc[0], clf_device_class_code.classes_, main2_df.device_class_code[0])

In [ ]:
def classes_proba(clf, X_test, y_test):
    """
    :param clf: classifyer
    :param X_test: test features
    :param y_test: test labels
    :return: vector of probability of truth class answer
    """
    probas = []
    for i, p_proba in enumerate(clf.predict_proba(X_test))
        probas.append(single_class_proba(p_proba, clf.classes_, y_test[i]))
    return probas

In [ ]:
dcc_feature = classes_proba(clf_device_class_code, full2_sparce_dummy, main2_df.device_class_code.fillna('NaN'))
print("Length: {}".format(dcc_feature))
dcc_feature

In [ ]:
def generate_features(clfs, X_test, y_tests):
    """
    :param clfs: list of classifyer
    :param X_test: test features
    :param y_test: list of test labels
    :return: matrix with new features for Groups Classification
    """
    features = []
    for i, clf in enumerate(clfs):
        feature_vec = classes_proba(clf, X_test, y_tests[i])
        print("Feature was generated. Length: {}".format(feature_vec))
        features.append(feature_vec)
    return features

In [ ]:
y_array = [
    main2_df.device_class_code.fillna('NaN'),
    main2_df.os_family_code.fillna('NaN'),
    main2_df.os_code.fillna('NaN'),
    main2_df.ua_class_code.fillna('NaN'),
    main2_df.ua_family_code.fillna('NaN'),
    main2_df.ua_version.fillna('NaN')
]

cls_array = [
    clf_device_class_code,
    clf_os_family_code,
    clf_os_code,
    clf_ua_class_code,
    clf_ua_family_code,
    clf_ua_version
]

human_features = generate_features(cls_array, full2_sparce_dummy, y_array)
pd.DataFrame(human_features)

In [ ]:
np.save('df/human_features.npy', np.array(human_features))
print("Human_features saved with size(Bytes): {}".format(os.stat('df/human_features.npy').st_size))

### Generate new human features based on probability of real class

In [ ]:
del(main2_df)
del(main2_data)
del(values2_data)
del(order2_data)
del(full2_sparce_dummy)

main2_bot_data = np.load('df/main2_bot_data.npy').tolist()
values2_bot_data = np.load('df/values2_bot_data.npy').tolist()
order2_bot_data = np.load('df/order2_bot_data.npy').tolist()

main2_bot_df = pd.DataFrame(main2_bot_data)

main2_bot_df

In [ ]:
l_parser = LogParser(log_folder='Logs/')
l_parser.reassign_orders_values(order2_bot_data, values2_bot_data)

full2_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=False)

In [ ]:
y_array = [
    main2_bot_df[1].fillna('NaN'),
    main2_bot_df[4].fillna('NaN'),
    main2_bot_df[3].fillna('NaN'),
    main2_bot_df[6].fillna('NaN'),
    main2_bot_df[7].fillna('NaN'),
    main2_bot_df[8].fillna('NaN')
]

cls_array = [
    clf_device_class_code,
    clf_os_family_code,
    clf_os_code,
    clf_ua_class_code,
    clf_ua_family_code,
    clf_ua_version
]

bot_features = generate_features(cls_array, full2_sparce_dummy, y_array)
pd.DataFrame(bot_features)

In [ ]:
np.save('df/bot_features.npy', np.array(bot_features))
print("Bot_features saved with size(Bytes): {}".format(os.stat('df/bot_features.npy').st_size))

## Superposition

In [ ]:
full_sp_features = pd.DataFrame(human_features + bot_features)
full_sp_features

In [ ]:
is_bot = pd.DataFrame(index=full_sp_features.index)
is_bot['is_bot'] = False
is_bot.loc[0:len(human_features) - 1,'is_bot'] = True
is_bot

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, make_scorer
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing

X_train, X_test, y_train, y_test = train_test_split(full_sp_features, is_bot.is_bot, test_size=0.33, random_state=42)

In [ ]:
%%time

smart_clf = LogisticRegression(C=100)
smart_clf.fit(X_train, y_train)
answer = clf.predict(X_test)

In [ ]:
filename = 'cls/groups_logreg_cls.joblib.pkl'
_ = joblib.dump(clf_device_class_code, filename, compress=9)

print("Model saved with size(Bytes): {}".format(os.stat(filename).st_size))

files_count = split_file(filename, 'parted-cls/groups_logreg_cls.joblib.pkl')

print('Splitted in {} files'.format(files_count))

In [ ]:
compare_frame = pd.DataFrame(y_test)
compare_frame['Predicted'] = answer
compare_frame['Success'] = compare_frame.Predicted == compare_frame.is_bot
compare_frame

In [ ]:
compare_frame.value_counts()

In [ ]:
first_kind_error = []
second_kind_error = []
for i, bot_prediction in enumerate(is_bot):
    cur_first_kind_error = False
    cur_second_kind_error = False
    if compare_frame.Predicted and not compare_frame.is_bot:
        cur_first_kind_error = True
    if not compare_frame.Predicted and compare_frame.is_bot:
        cur_second_kind_error = True
    first_kind_error.append(cur_first_kind_error)
    second_kind_error.append(cur_second_kind_error)    

compare_frame['first_kind_error'] = first_kind_error
compare_frame['second_kind_error'] = second_kind_error

print('Ошибка первого рода (когда мы принимаем нормального пользователя за бота): {}'.format(sum(first_kind_error) / y_test.shape[0]))
print('Ошибка второго рода (когда мы принимаем бота за нормального пользователя): {}'.format(sum(second_kind_error) / y_test.shape[0]))